In [1]:
import numpy as np
from scipy.stats import norm

In [2]:

# Set a random seed for reproducibility
np.random.seed(42)
# Represent the die:
dice_faces = np.array([1, 2, 3, 4, 5, 6])
# Initialize the probabilities of each face of the die
probabilities = np.array([1, 1, 1, 1, 3, 1])
probabilities = probabilities / probabilities.sum()  # Normalize
print("the true probabilities are:", probabilities)
# Calculate the true expected value based on initialized probabilities
true_expected_value = np.sum(probabilities * dice_faces)
print("True expected value of the die:", round(true_expected_value, 2))

the true probabilities are: [0.125 0.125 0.125 0.125 0.375 0.125]
True expected value of the die: 3.88


In [5]:
# Function to simulate rolling the biased die
def roll_die(probabilities, num_rolls=100):
    return np.random.choice(dice_faces, size=num_rolls, p=probabilities)

# Function to perform the E-step
def expectation(rolls):
    counts = np.zeros(6)
    for value in rolls:
        counts[value - 1] += 1
    return counts / len(rolls)

# Function to perform the M-step
def maximization(expected_frequencies):
    return expected_frequencies / expected_frequencies.sum()


In [4]:
# Simulate rolling the die
rolls = roll_die(probabilities)

# EM algorithm with hypothesis testing for convergence
expected_values = []

for _ in range(100):  # Max iterations
    expected_frequencies = expectation(rolls)
    probabilities = maximization(expected_frequencies)
    expected_value = np.sum(probabilities * dice_faces)
    expected_values.append(expected_value)
    
    if len(expected_values) > 1:
        # Calculate change in expected value and perform z-test
        change = expected_values[-1] - expected_values[-2]
        if len(expected_values) > 2:
            mean_change = np.mean(np.diff(expected_values))
            std_change = np.std(np.diff(expected_values), ddof=1)
            if std_change > 0:
                z_score = mean_change / (std_change / np.sqrt(len(expected_values) - 1))
                p_value = 2 * norm.sf(abs(z_score))  # Two-tailed test
                
                # Stop if p-value is above 0.05, indicating stabilization
                if p_value > 0.05:
                    print(f"Algorithm stops after {_+1} iterations with a p-value of {p_value:.4f}")
                    break

print("Final estimated probabilities:", probabilities)
print("Final expected value of the die:", round(expected_value, 2))

Final estimated probabilities: [0.1253  0.12516 0.12551 0.12337 0.37754 0.12312]
Final expected value of the die: 3.87
